In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.set_num_threads(8)

# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            # 'dataset': "CIFAR100",
            'dataset': "Drug",
            'fairness_agg': 'mean',
            'random_class_idx': False,

            # benchmark
            'seed': seed,
            'num_tasks': 3,
            'epochs_per_task': 25,
            'per_task_examples': np.inf,
            # 'per_task_examples': 10000,
            'per_task_memory_examples': 64,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 1.,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.BCEWithLogitsLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.000,
            'lambda': .0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=Drug/seed=10_epoch=25_lr=0.001_alpha=0.0_tau=1.0


In [2]:
from datasets import BiasedMNIST, Drug

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                random_class_idx=True)
    input_dim = (3, 28, 28)
elif params['dataset'] == 'Drug':        
    benchmark = Drug(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = params['random_class_idx'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (12)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)


In [3]:
from algorithms.icarl import iCaRL
from metrics import FairMetricCollector
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

# from backbones import ResNet18
# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

algorithm = iCaRL(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])


iCaRL


In [4]:
benchmark.drug_train.data.shape[-1]

12

In [5]:
from trainers.baselines import BaseContinualTrainer as ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [6]:
# CIFAR10: self.dataset.data.shape=(50000, 32, 32, 3)


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.6344866071428572, 'loss': 0.0, 'std': 0.04520089285714285, 'EER': -1, 'EO': [0.23469387755102045, 0.3358974358974359], 'DP': -1, 'accuracy_s0': 0.6346153846153846, 'accuracy_s1': 0.584013605442177, 'classwise_accuracy': {0: array([ 87, 128]), 1: array([33, 56])}, 'DP_ingredients': {'class_pred_count_s0': {1: 35, 0: 21}, 'class_pred_count_s1': {0: 89, 1: 39}, 'class_pred_count': {1: 74, 0: 110}, 'count_s0': 56, 'count_s1': 128, 'count': 184}}
[2] Eval metrics for task 1 >> {'accuracy': 0.6305803571428572, 'loss': 0.0, 'std': 0.04129464285714285, 'EER': -1, 'EO': [0.22448979591836737, 0.3358974358974359], 'DP': -1, 'accuracy_s0': 0.6346153846153846, 'accuracy_s1': 0.5789115646258504, 'classwise_accuracy': {0: array([ 86, 128]), 1: array([33, 56])}, 'DP_ingredients': {'class_pred_count_s0': {1: 35, 0: 21}, 'class_pred_count_s1': {0: 88, 1: 40}, 'class_pred_count': {0: 109, 1: 75}, 'co

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.608, 0.   , 0.   ],
       [0.493, 0.43 , 0.   ],
       [0.523, 0.187, 0.34 ]])

In [9]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

0.47307315080648343

In [10]:
[np.round(x, 3) for x in metric_manager_callback.meters['EO'].compute_overall()]

[0.278, 0.242, 0.174]

In [11]:
np.mean(metric_manager_callback.meters['EO'].compute_overall())

0.23107857459500827

In [12]:
[np.round(x, 3) for x in metric_manager_callback.meters['DP'].compute_overall()]

[0.146, 0.079, 0.06]

In [13]:
np.mean(metric_manager_callback.meters['DP'].compute_overall())

0.09500998254118125